In [1]:
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import Petna as pt
matplotlib.rcParams['figure.figsize'] = (50,50)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
pd.set_option('display.max_rows', 2000)
%reload_ext autoreload
%autoreload 2

In [2]:
xls = pd.ExcelFile('2019 County Health Rankings Data - v2.xls')
sheets = xls.sheet_names
sheets

['Introduction',
 'Outcomes & Factors Rankings',
 'Outcomes & Factors SubRankings',
 'Ranked Measure Data',
 'Ranked Measure Sources & Years',
 'Additional Measure Data',
 'Addtl Measure Sources & Years']

In [3]:
# pt.missingvalues(sheets[5],xls)

In [4]:
rmd = xls.parse(sheets[3],header=1)
amd = xls.parse(sheets[5],header=1)
amd = amd.drop(columns=['State','County'])

## Creating our master dataFrame

In [5]:
df = rmd.merge(amd, on='FIPS')
# Focusing on percentage and rates as opposed to absolute values
for kw in ['95%','Quartile','#','Unreliable']:
    df = df.drop(columns=[x for x in df.columns if kw in x])
df = df.set_index('FIPS')

In [6]:
# Columns title formatting
subs = [(' ', '_'),('.',''),("'",""),('™', ''), ('®',''),
        ('+','plus'), ('½','half'), ('-','_'), ('<','under'), ('%','percent'), ('/', '_or_') 
       ]
def col_formatting(col):
    for old, new in subs:
        col = col.replace(old,new)
    return col

df.columns = [col_formatting(col) for col in df.columns]

## Replacing NaN values by state average

MV_Mortality_Rate, YPLL and %Alcohol Impaired car crashed NaN values have been replaced by the state mean. Average_Daily_PM25 Nan are replaced by 0 as only the whole of Alska and Hawai have Nan.

In [7]:
df['MV_Mortality_Rate'] = df['MV_Mortality_Rate'].fillna(df.groupby(by='State')['MV_Mortality_Rate'].transform('mean'))
df['Years_of_Potential_Life_Lost_Rate'] = df['Years_of_Potential_Life_Lost_Rate'].fillna(df.groupby(by='State')['Years_of_Potential_Life_Lost_Rate'].transform('mean'))
df['percent_Alcohol_Impaired'] = df['percent_Alcohol_Impaired'].fillna(df.groupby(by='State')['percent_Alcohol_Impaired'].transform('mean'))
df['Average_Daily_PM25'] = df['Average_Daily_PM25'].fillna(0)

In [8]:
# df.to_csv('df.csv')